# Test notebook for no automated Differential Privacy (DP) data type
## Data Scientist
### syft version: 0.6.0

In [1]:
import syft as sy
print(sy.__version__)
import torch
import numpy as np

0.6.0


**DS: login to DO's domain node**

In [2]:
ds_client = sy.login(
    # url="http://96.20.213.92",
    url = "192.168.0.77",
    email="shu00.song@gmail.com",
    password="1234",
    port=8081
)

Connecting to 192.168.0.77... done! 	 Logging into free_node... done!


**Look for available datasets**

In [3]:
ds_client.store.pandas

,ID,Tags,Description,object_type
0,<UID: a2e30a3b528b4ff898053bf7f6ea84dd>,[],,<class 'torch.Tensor'>


In [4]:
ds_client.datasets # Doesn't work for not autoDP data

In [8]:
data_th_ptr = ds_client.store[0] 
print(data_th_ptr)

<TensorPointer -> free_node:a2e30a3b528b4ff898053bf7f6ea84dd>


In [10]:
average = data_th_ptr.float().mean() # Works on torch tensors
print(average)

<TensorPointer -> free_node:c76bd8358346445c849b2653b8b0fa54>


In [11]:
average.get() # I do not have permission, which is normal

[2022-06-22T16:31:30.849525-0400][CRITICAL][logger]][14047] You do not have permission to .get() Object with ID: <UID: c76bd8358346445c849b2653b8b0fa54> on node free_node Please submit a request.


AuthorizationException: You do not have permission to .get() Object with ID: <UID: c76bd8358346445c849b2653b8b0fa54> on node free_node Please submit a request.

**DS: make requests to access result values**

In [12]:
average.request(reason = "Please approve!")

**Access result values after receiving permission from DO**

In [13]:
avg_value = average.get()

In [14]:
print(avg_value)

tensor(0.5000)


In [18]:
# After enabling the request handler (DO's side)
max_value_ptr = data_th_ptr.float().amax()

max_value = max_value_ptr.get(
    request_block=True,
    reason="download the maximum value",
    timeout_secs=10,
)

print(max_value)

[2022-06-22T16:39:21.478713-0400][CRITICAL][logger]][14047] UnknownPrivateException has been triggered.


None


## Using remote Torch
Check [here](https://github.com/OpenMined/courses/blob/foundations-of-private-computation/federated-learning/duet_fl/Duet_FL_Data_Scientist.ipynb) for concrete example.  
> remote_torch = < ds_client >.torch

In [27]:
ds_client.torch # Does exist still

Module:
	.Parameter -> <syft.ast.klass.Class object at 0x7f1b666c4b20>
	.Module -> <syft.ast.klass.Class object at 0x7f1b665d2520>
	.Conv2d -> <syft.ast.klass.Class object at 0x7f1b665d2a00>
	.Dropout2d -> <syft.ast.klass.Class object at 0x7f1b665d2e20>
	.Linear -> <syft.ast.klass.Class object at 0x7f1b665d81c0>
	.functional -> Module:
		.relu -> <syft.ast.callable.Callable object at 0x7f1b665d8ac0>
		.gelu -> <syft.ast.callable.Callable object at 0x7f1b665d8b20>
		.max_pool2d -> <syft.ast.callable.Callable object at 0x7f1b665d8b80>
		.log_softmax -> <syft.ast.callable.Callable object at 0x7f1b665d8be0>
		.cosine_embedding_loss -> <syft.ast.callable.Callable object at 0x7f1b665e29a0>
		.ctc_loss -> <syft.ast.callable.Callable object at 0x7f1b665e2a00>
		.hinge_embedding_loss -> <syft.ast.callable.Callable object at 0x7f1b665e2ac0>
		.l1_loss -> <syft.ast.callable.Callable object at 0x7f1b665e2b20>
		.margin_ranking_loss -> <syft.ast.callable.Callable object at 0x7f1b665e2be0>
		.mse_lo

**Creating a model compatible with remote computation:**  
Using sy.Module

In [19]:
# Doesn't work anymore
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        self.linear = self.torch_ref.nn.Linear(in_dim, out_dim)

    def forward(self, x):
        x = self.linear(x)
        return x   

AttributeError: module 'syft' has no attribute 'Module'